In [1]:
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

import numpy
import pandas as pd

In [4]:
corpus = ["Python is popular in machine learning",
         "Distributed system is important in big data analysis",
        "Machine learning is theoretical foundation of data mining",
        "Learning Python is fun",
        "Playing soccer is fun",
        "Many data scientists like playing soccer",
        "Chinese men's soccer team failed again",
        "Thirty two soccer teams enter World Cup finals"]

vectorizer = CountVectorizer(min_df=1, stop_words="english")
data = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

pd.DataFrame(data.toarray(), index=corpus, columns=vectorizer.get_feature_names_out()).head(10)

,analysis,big,chinese,cup,data,distributed,enter,failed,finals,foundation,...,playing,popular,python,scientists,soccer,team,teams,theoretical,thirty,world
Python is popular in machine learning,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
Distributed system is important in big data analysis,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Machine learning is theoretical foundation of data mining,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
Learning Python is fun,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Playing soccer is fun,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
Many data scientists like playing soccer,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
Chinese men's soccer team failed again,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,0
Thirty two soccer teams enter World Cup finals,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,1,0,1,0,1,1


In [ ]:
"""Singular value decomposition and LSA"""
model = TruncatedSVD(2)
data_n = model.fit_transform(data)
data_n = Normalizer(copy=False).fit_transform(data_n)
data_n

In [ ]:
pd.DataFrame(data_n, index = corpus, columns = ["component_1", "component_2"])

In [ ]:
xs = data_n[:,0]
ys = data_n[:,1]
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.figure(figsize=(4,4))
ax = plt.gca()
ax.set_xlim([-1, 2])
ax.set_ylim([-1, 2])
plt.scatter(xs, ys)
plt.xlabel('First principal component')
plt.ylabel('Second principal component')
plt.title('Plot of points agains LSA principal components')
plt.show()


In [ ]:
similarity = numpy.asarray(numpy.asmatrix(data_n) * numpy.asmatrix(data_n).T)
pd.DataFrame(similarity, index = corpus, columns = corpus).head(10)

In [ ]:
sns.heatmap(similarity,cmap='Reds')

In [ ]:
pd.DataFrame(model.components_,index=['component_1','component_2'],columns=vectorizer.get_feature_names()).T